In [0]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
import os
from csv import reader
import re
import pathlib
import sys

class DummyDataset(Dataset):
    def __init__(self):
        super().__init__()

    def __len__(self):
        return 96

    def __getitem__(self, index):
        bert_embed = torch.tensor(np.zeros((1,300))).double()
        glove_cove_embed = torch.tensor(np.zeros((1,600))).double()
        return (bert_embed, glove_cove_embed), torch.tensor(0).double()

    def get_map(self):
        return self.id_map


# sample = DummyDataset()
# # #print(sample.__getitem__(0))
# # train_loader =  torch.utils.data.DataLoader(sample, batch_size=32, 
# #                                                shuffle=True, num_workers=1)


In [45]:
######################################################################
#
#   Architecture is as follows
#   1. Bert word encoding + CNN with max pooling for sent.embed
#   2. Glove & CoVe word Encoding  + CNN with attention for sent.embed
#   3. (1) and (2) concated and fed to MLP for classification (1-sugg)

class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.l0 = nn.Linear(900,300) # BERT-> CNN (300), G,C->CNN (600)
        self.l1 = nn.Linear(300,300)
        self.l3 = nn.Linear(300,1)
        self.dp = nn.Dropout(p=0.5)

    def forward(self,x):
        # print(x.shape)
        # print(x.type())
        x = self.dp(F.sigmoid(self.l0(x)))
        # print(x.shape)
        x = self.dp(F.sigmoid(self.l1(x)))
        # print(x.shape)
        x = F.sigmoid(self.l3(x))
        # print(x.shape)
        return x


# class SuggestionClassifier(nn.Module):
#     def __init_(self):
#         # path all the models together
#         # BERT->CNN + CNN->ATT
#         super(SuggestionClassifier,self).__init__()
#         # self.CNN_b = CNN_BERT()
#         # self.CNN_gc = CNN_GC()
#         self.MLP = MLP()


#     def forward(self, x):
#         # bert_x = self.CNN_b(x[0])
#         # gc_x = self.CNN_gc(x[1])
#         bert_x = x[0]
#         gc_x = x[1]
        
#         out = torch.cat(bert_x,gc_x) # TODO : ensure dimensions align (1x300, 1x600)
#         out = self.MLP(out) # Dim : 1x900
#         return out


#############################
# HELPER FUNCTIONS

def train(model, train_loader, valid_loader, test_loader, foldId, numEpochs=5):
    '''
    Args:

    Ret: 
    '''

    model.train()
    val_loss_min = None
    val_loss_min_delta = 0
    val_patience = 0
    val_loss_counter = 0

    # TODO : 10-fold cross validation (build 10 models)
    # TODO : Ensemble, pick top three models based on train loss
    # TODO : Voting Mechanism to evaluate on Validation data
    
    # for each fold
    #     for each epoch
    #       min_batch_train(model)
    #       val_loss = model(trial_data)
    #       early_stopping(val_loss)
    #     models += models
    #     three_models = select_best_3(models) based on train_loss
        
    for epoch in range(numEpochs):
        avg_loss = 0.0
        for batch_num, (feats, labels) in enumerate(train_loader):

            #print(feats[0].size())
            #print(feats[0].permute(2,1,0).size())
            feats = torch.cat((feats[0].permute(2,1,0),feats[1].permute(2,1,0))).permute(2,1,0)

            feats,labels = feats.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(feats)
            print(outputs.shape)
            loss = criterion(outputs, labels.view(BATCH_SIZE,1).long())
            print(loss)
            loss.backward()
            optimizer.step()
            print("Optimizer Complete")

            avg_loss += loss.item()
            print(avg_loss)

            if batch_num % 50 == 49:
                print('Fold: {}\t Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(foldId+1, epoch+1, batch_num+1, avg_loss/50))
                avg_loss = 0.0

            torch.cuda.empty_cache()
            del feats
            del labels

        model_name = "jessi_A_fold_"+str(foldId+1)+"_epoch_"+str(epoch+1)+".pt"
        print(model_name)
        torch.save(model.state_dict(), model_name)

        val_loss, val_acc = validate(model, valid_loader)
        test_loss, test_acc = validate(model, test_loader)
        print('Epoch{:.4f}\tVal Loss: {:.4f}\tVal Accuracy: {:.4f}\tTest Loss: {:.4f}\tTest Accuracy: {:.4f}'.format(epoch, val_loss, val_acc, test_loss, test_acc))

        # Early stopping
        if val_loss_min is None:
            val_loss_min = val_loss
        elif val_loss_min >= (val_loss + val_loss_min_delta) :
            val_loss_counter += 1
            if (val_loss_counter > val_patience) :
                print("Validation Loss: {}\t Lowest Validation Loss {}\t".format(val_loss, val_loss_min))
                print("Training stopped early, Epoch :"+str(epoch))
                break
        else:
            val_loss_min = val_loss

    return test_acc, model_name


def validate(model, valid_loader):
    '''
    Args:

    Ret:
    '''

    model.eval()
    test_loss = []
    accuracy = 0
    total = 0

    for batch_num, (feats, labels) in enumerate(valid_loader):
        feats = torch.cat((feats[0].permute(2,1,0),feats[1].permute(2,1,0))).permute(2,1,0)
        feats,labels = feats.to(device), labels.to(device)
        outputs = model(feats)

        pred_labels = torch.round(outputs)
        pred_labels = pred_labels.view(-1)

        loss = criterion(outputs, labels.view(BATCH_SIZE,1).long())
        accuracy += torch.sum(torch.eq(pred_labels, labels)).item()
        total += len(labels)
        test_loss.extend([loss.item()]*feats.size()[0])

        #torch.cuda.empty_cache()
        del feats
        del labels

    model.train()
    return np.mean(test_loss), accuracy/total
    #return 0, accuracy/total


def test(models, test_loader, id_map=0):
    '''
    Args:

    Ret:
    '''

    test_loss = []
    accuracy = 0
    total = 0
    
    outcomes = []
    
    for batch_num, (feats, labels) in enumerate(test_loader):
        #print(feats[0].size())
        feats = torch.cat((feats[0].permute(2,1,0),feats[1].permute(2,1,0))).permute(2,1,0)
        feats, labels = feats.to(device), labels.to(device)
        predictions = torch.tensor(np.zeros(len(labels))).to(device)

        for model in models:
            model = model.double()
            model.eval()
            outputs = model(feats)
            pred_labels = torch.round(outputs)
            predictions += pred_labels.view(-1)
            model.train()

        predictions = torch.round(predictions/3)
        accuracy += torch.sum(torch.eq(predictions, labels)).item()
        total += len(labels)       
        # outcomes += [ [id_map(test_loader.dataset.samples[batch_num * BATCH_SIZE + i]),predictions[i]] for i in range(len(predictions)) ]
            
        del feats
        del labels
    
    # f = open("Suggestion.csv", "w", newline="")
    # writer = csv.writer(f)
    # writer.writerows(outcomes)
    # f.close()
    
    #return np.mean(test_loss), accuracy/total
    return -1, accuracy/total


#########################
# HYPER PARAMETER TUNING

NUM_EPOCHS = 1
WEIGHT_DECAY = 3
BATCH_SIZE = 32
NFOLDS = 10
NUM_WORKERS = 1
LEARNING_RATE = 0.1 # TODO : Experiment, as LR not given

########################
# MODEL CREATION

def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        torch.nn.init.kaiming_normal_(m.weight.data)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MLP()
# TODO:  WEIGHT INIT
model.apply(init_weights)
model = model.double()
model.train()
model.to(device)

#######################
# OPTIMIZATION

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adadelta(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

########################
# DATA LOADING

# valid_dataset = SuggestionDataset(load_data(filename="SubtaskA_Trial_Test_Labeled.csv"))
valid_dataset = DummyDataset()
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, 
                                               shuffle=False, num_workers=NUM_WORKERS)

# test_dataset = SuggestionDataset(load_data(filename="SubtaskA_EvaluationData_labeled.csv"), mode=0)
# test_id_map = test_dataset.get_map()
test_dataset = DummyDataset()
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, 
                                               shuffle=False, num_workers=NUM_WORKERS)

# data = load_data()
# data_folds = create_folds(data)
model_perfs = []
model_names = []
for i in range(NFOLDS):
    # train, test = create_cross_val_train_test(data_folds,i)
    # train_dataset = SuggestionDataset(train)
    train_dataset = DummyDataset()
    train_loader =  torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, 
                                               shuffle=True, num_workers=NUM_WORKERS)
    # test_dataset = SuggestionClassifier(test)
    test_dataset = DummyDataset()
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, 
                                               shuffle=False, num_workers=NUM_WORKERS)

    test_loss, model_name = train(model, train_loader, valid_loader, test_loader, i, NUM_EPOCHS)

    model_perfs.append(test_loss)
    model_names.append(model_name)


print(model_perfs)
print(model_names)
best_three_model_idx = np.flip(np.argsort(model_perfs))[0:3]
models = []
for idx in best_three_model_idx:
    # load the model
    #temp_model = SuggestionClassifier()
    temp_model = MLP()
    temp_model.load_state_dict(torch.load(model_names[idx]))
    models.append(temp_model)
# testing
print(len(models))
test(models, test_loader)


torch.Size([32, 1, 1])
tensor(0., dtype=torch.float64, grad_fn=<NllLoss2DBackward>)
Optimizer Complete
0.0
torch.Size([32, 1, 1])
tensor(0., dtype=torch.float64, grad_fn=<NllLoss2DBackward>)
Optimizer Complete
0.0
torch.Size([32, 1, 1])
tensor(0., dtype=torch.float64, grad_fn=<NllLoss2DBackward>)
Optimizer Complete
0.0
jessi_A_fold_1_epoch_1.pt


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch0.0000	Val Loss: 0.0000	Val Accuracy: 1.0000	Test Loss: 0.0000	Test Accuracy: 1.0000
torch.Size([32, 1, 1])
tensor(0., dtype=torch.float64, grad_fn=<NllLoss2DBackward>)
Optimizer Complete
0.0
torch.Size([32, 1, 1])
tensor(0., dtype=torch.float64, grad_fn=<NllLoss2DBackward>)
Optimizer Complete
0.0
torch.Size([32, 1, 1])
tensor(0., dtype=torch.float64, grad_fn=<NllLoss2DBackward>)
Optimizer Complete
0.0
jessi_A_fold_2_epoch_1.pt
Epoch0.0000	Val Loss: 0.0000	Val Accuracy: 1.0000	Test Loss: 0.0000	Test Accuracy: 1.0000
torch.Size([32, 1, 1])
tensor(0., dtype=torch.float64, grad_fn=<NllLoss2DBackward>)
Optimizer Complete
0.0
torch.Size([32, 1, 1])
tensor(0., dtype=torch.float64, grad_fn=<NllLoss2DBackward>)
Optimizer Complete
0.0
torch.Size([32, 1, 1])
tensor(0., dtype=torch.float64, grad_fn=<NllLoss2DBackward>)
Optimizer Complete
0.0
jessi_A_fold_3_epoch_1.pt
Epoch0.0000	Val Loss: 0.0000	Val Accuracy: 1.0000	Test Loss: 0.0000	Test Accuracy: 1.0000
torch.Size([32, 1, 1])
tensor(0., dt

(-1, 1.0)